# 1. model 생성을 위한 units = 0이 아닌 데이터만 뽑아서 만든다

## 1-1 전체 데이터 load

In [1]:
from jw_package import *
from filling_missing import *
from functools import *

import configuration completed !
train + key + weather merging started
train + key + weather merging finished
function configuration completed ! 
Good to go !


In [2]:
each_station = []

for station_nbr in range(1,21):
    file_path = '../data/weather_best_close_final/{:02}.csv'.format(station_nbr)
    station = pd.read_csv(file_path,index_col=0)
    station.dropna(axis=0,how='any',inplace=True)
    each_station.append(station)

weather = reduce((lambda x,y : x.append(y)),each_station)

In [3]:
each_station = []

for station_nbr in range(1,21):
    file_path = '../data/weather_best_close_final/{:02}.csv'.format(station_nbr)
    station = pd.read_csv(file_path,index_col=0)
    station.dropna(axis=0,how='any',inplace=True)
    each_station.append(station)

weather = reduce((lambda x,y : x.append(y)),each_station)

In [4]:
sales = pd.read_csv('../data/basic/train.csv')
keys = pd.read_csv('../data/basic/key.csv')

In [5]:
data = sales.merge(keys).merge(weather)

In [6]:
data.head()

,date,store_nbr,item_nbr,units,station_nbr,tmax,tmin,tavg,depart,dewpoint,...,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,year,month,day,codesum
0,2012-01-01,1,1,0,1,52.0,31.0,42.0,10.0,36.0,...,0.05,29.78,29.92,3.6,20.0,4.6,2012,1,1,RA FZFG BR
1,2012-01-01,1,2,0,1,52.0,31.0,42.0,10.0,36.0,...,0.05,29.78,29.92,3.6,20.0,4.6,2012,1,1,RA FZFG BR
2,2012-01-01,1,3,0,1,52.0,31.0,42.0,10.0,36.0,...,0.05,29.78,29.92,3.6,20.0,4.6,2012,1,1,RA FZFG BR
3,2012-01-01,1,4,0,1,52.0,31.0,42.0,10.0,36.0,...,0.05,29.78,29.92,3.6,20.0,4.6,2012,1,1,RA FZFG BR
4,2012-01-01,1,5,0,1,52.0,31.0,42.0,10.0,36.0,...,0.05,29.78,29.92,3.6,20.0,4.6,2012,1,1,RA FZFG BR


## 1-2 tmp 데이터로 copy후 units = 0이 아닌 것만 뽑아서 model 생성

In [7]:
tmp = data.copy()

In [8]:
tmp = tmp[tmp['units']!=0]

In [9]:
#etc = ['day','month','year','station_nbr']
category = ['sunrise','sunset','codesum']

for each in category:
    tmp.drop(each,axis=1,inplace=True)

In [10]:
del tmp['tavg']
del tmp['sealevel']
del tmp['wetbulb']
del tmp['stnpressure']
del tmp['tmin']
del tmp['tmax']
del tmp['dewpoint']
del tmp['avgspeed']
del tmp['resultspeed']
del tmp['depart']
del tmp['heat']
del tmp['cool']
del tmp['snowfall']
del tmp['preciptotal']
del tmp['resultdir']

In [11]:
tmp.reset_index(drop = True, inplace = True)

In [12]:
tmp.tail()

,date,store_nbr,item_nbr,units,station_nbr,year,month,day
118659,2014-10-29,35,66,14,5,2014,10,29
118660,2014-10-30,35,16,16,5,2014,10,30
118661,2014-10-30,35,66,6,5,2014,10,30
118662,2014-10-31,35,16,18,5,2014,10,31
118663,2014-10-31,35,66,10,5,2014,10,31


# 2. 이부분은 codesum_v1.csv 데이터 load

In [13]:
codesum = pd.read_csv('../data/weather_v3.csv', index_col = 0)

In [14]:
codesum = codesum.merge(keys, on = 'station_nbr' )

In [15]:
codesum = codesum.loc[:,['date','store_nbr','is_holiday','codesum', 'dayofweek']]

In [16]:
def codesum_ch(codesum):
    if codesum == 'HZ':
        return 'AN'
    elif codesum == 'BR':
        return 'AN'
    elif codesum == 'FG,BR':
        return 'AN'
    elif codesum == 'FG,FG,BR':
        return 'AN'
    elif codesum == 'RA,BR':
        return 'RA'
    elif codesum == 'RA,BR,HZ':
        return 'RA'
    elif codesum == 'RA,BR,TS':
        return 'RA'
    elif codesum == 'RA,HZ,BR':
        return 'RA'
    elif codesum == 'TS,RA,RA':
        return 'RA'
    elif codesum == 'TS,TS,RA':
        return 'RA'
    elif codesum == 'TS,RA':
        return 'RA'
    elif codesum == 'RA,TS':
        return 'RA'
    elif codesum == 'TS,RA,BR':
        return 'RA'
    elif codesum == 'BR,HZ':
        return 'AN'
    else:
        return codesum

In [17]:
codesum['codesum'] = codesum['codesum'].apply(codesum_ch)

In [18]:
tmp = tmp.merge(codesum, on = ['date','store_nbr'])

In [19]:
tmp.head()

,date,store_nbr,item_nbr,units,station_nbr,year,month,day,is_holiday,codesum,dayofweek
0,2012-01-01,1,9,29,1,2012,1,1,holiday,AN,Sun
1,2012-01-01,1,28,2,1,2012,1,1,holiday,AN,Sun
2,2012-01-01,1,51,1,1,2012,1,1,holiday,AN,Sun
3,2012-01-02,1,9,60,1,2012,1,2,holiday_work,MO,Mon
4,2012-01-02,1,28,5,1,2012,1,2,holiday_work,MO,Mon


## 2-1. codesum에서 holiday 컬럼, codesum 컬럼 category변환 함수 
- holiday 이면 1 아니면 0
- codesum 'Not_MO'이면 1 MO 이면 0 

In [20]:
def cate_holiday(holiday):
    if holiday == 'holiday' or holiday == 'holiday_work':
            return 1
    else:
        return 0
def cate_codesum(codesum):
    if codesum == 'MO':
        return 0 
    else:
        return 1
    
tmp['is_holiday'] = tmp['is_holiday'].apply(cate_holiday)
#tmp['codesum'] = tmp['codesum'].apply(cate_codesum)

In [21]:
tmp.head()

,date,store_nbr,item_nbr,units,station_nbr,year,month,day,is_holiday,codesum,dayofweek
0,2012-01-01,1,9,29,1,2012,1,1,1,AN,Sun
1,2012-01-01,1,28,2,1,2012,1,1,1,AN,Sun
2,2012-01-01,1,51,1,1,2012,1,1,1,AN,Sun
3,2012-01-02,1,9,60,1,2012,1,2,1,MO,Mon
4,2012-01-02,1,28,5,1,2012,1,2,1,MO,Mon


# 3. model 생성
- formula = 'units ~ C(store_nbr) + C(item_nbr)+ scale(cool) + scale(heat) + scale(preciptotal) + scale(snowfall) + is_holiday'

In [22]:
len(tmp)

118664

In [23]:
formula = 'units ~ C(store_nbr) + C(item_nbr) + C(year) + C(day) +C(codesum)'
model = sm.OLS.from_formula(formula, data= tmp)
result = model.fit()

In [24]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  units   R-squared:                       0.600
Model:                            OLS   Adj. R-squared:                  0.599
Method:                 Least Squares   F-statistic:                     940.4
Date:                Tue, 13 Mar 2018   Prob (F-statistic):               0.00
Time:                        16:15:53   Log-Likelihood:            -5.7477e+05
No. Observations:              118664   AIC:                         1.150e+06
Df Residuals:                  118474   BIC:                         1.152e+06
Df Model:                         189                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              36.6747    

## 3-1. cross validation 확인

In [25]:
len(tmp)

118664

In [26]:
X = tmp.loc[:, ['store_nbr', 'item_nbr', 'year', 'day','codesum']]
y = tmp.loc[:, ['units']]

In [27]:
from sklearn.model_selection import KFold

cv = KFold(n_splits=4, shuffle=True, random_state=0)
for train_index, test_index in cv.split(X):
    print("test index :", test_index, len(test_index))
    print("." * 80)
    print("train index:", train_index, len(train_index))
    print("=" * 80)

    X_train = X.loc[train_index, :]
    X_test = X.loc[test_index, :]
    y_train = y.loc[train_index, :]
    y_test = y.loc[test_index, :]
    
    tmp_data = pd.concat([X_train, y_train], axis = 1)
    formula = 'units ~ C(store_nbr) + C(item_nbr)+ C(year) + C(day) + C(codesum)'
    
    model = sm.OLS.from_formula(formula, data= tmp_data)
    result = model.fit()
    
    
    y_hat = result.predict(X_test)
    
    
    
    print(((y_hat - y_hat.mean())**2).sum()  / ((y_test-y_test.mean())**2).sum() )

test index : [     3      6      9 ..., 118659 118660 118663] 29666
................................................................................
train index: [     0      1      2 ..., 118658 118661 118662] 88998
units    0.465965
dtype: float64
test index : [     1      8     11 ..., 118652 118655 118662] 29666
................................................................................
train index: [     0      2      3 ..., 118660 118661 118663] 88998
units    0.694185
dtype: float64
test index : [     0      4      5 ..., 118642 118653 118658] 29666
................................................................................
train index: [     1      2      3 ..., 118661 118662 118663] 88998
units    0.601802
dtype: float64
test index : [     2     10     13 ..., 118641 118649 118661] 29666
................................................................................
train index: [     0      1      3 ..., 118660 118662 118663] 88998
units    0.705362
dtype: float64


# 4. Outlier 제거 코드

In [28]:
len(tmp)

118664

In [50]:
res = result.resid_pearson
index = result.resid.index
outlier_index = abs(res) >= 2

row = pd.Series(index=index, data=res)
drop_index = row[outlier_index].index
len(drop_index)

tmp.drop(list(drop_index),inplace=True)
tmp.reset_index(drop = True, inplace = True)

X = tmp.loc[:, ['store_nbr', 'item_nbr', 'year', 'day', 'codesum']]
y = tmp.loc[:, ['units']]

## 4-1. 2차 제거 후 cross_val 확인
- 위 코드 두번 실행

In [51]:
len(tmp)

88851

In [52]:
from sklearn.model_selection import KFold

cv = KFold(n_splits=4, shuffle=True, random_state=0)
for train_index, test_index in cv.split(X):
    print("test index :", test_index, len(test_index))
    print("." * 80)
    print("train index:", train_index, len(train_index))
    print("=" * 80)

    X_train = X.loc[train_index, :]
    X_test = X.loc[test_index, :]
    y_train = y.loc[train_index, :]
    y_test = y.loc[test_index, :]
    
    tmp_data = pd.concat([X_train, y_train], axis = 1)
    formula = 'units ~ C(store_nbr) + C(item_nbr)+ C(year) + C(day)+ C(codesum)'
    
    model = sm.OLS.from_formula(formula, data= tmp_data)
    result = model.fit()
    
    
    y_hat = result.predict(X_test)
    
    
    
    print(((y_hat - y_hat.mean())**2).sum()  / ((y_test-y_test.mean())**2).sum() )

test index : [    1     3     8 ..., 88838 88842 88849] 22213
................................................................................
train index: [    0     2     4 ..., 88847 88848 88850] 66638
units    0.912949
dtype: float64
test index : [    0     5     6 ..., 88843 88845 88846] 22213
................................................................................
train index: [    1     2     3 ..., 88848 88849 88850] 66638
units    0.905179
dtype: float64
test index : [    4     7    15 ..., 88844 88847 88850] 22213
................................................................................
train index: [    0     1     2 ..., 88846 88848 88849] 66638
units    0.914732
dtype: float64
test index : [    2    10    13 ..., 88832 88840 88848] 22212
................................................................................
train index: [    0     1     3 ..., 88847 88849 88850] 66639
units    0.919054
dtype: float64


# 4-2. 현재모델

In [53]:
formula = 'units ~ C(store_nbr) + C(item_nbr)+ C(year) + C(day) + C(codesum)'
model = sm.OLS.from_formula(formula, data= tmp)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  units   R-squared:                       0.913
Model:                            OLS   Adj. R-squared:                  0.913
Method:                 Least Squares   F-statistic:                     4927.
Date:                Tue, 13 Mar 2018   Prob (F-statistic):               0.00
Time:                        16:39:56   Log-Likelihood:            -3.0614e+05
No. Observations:               88851   AIC:                         6.127e+05
Df Residuals:                   88661   BIC:                         6.144e+05
Df Model:                         189                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              13.8287    

In [54]:
# 정규화  result1 = Ridge 모형
result1 = model.fit_regularized(alpha=0.1, L1_wt=0)

# 정규화  result2 = Lasso 모형
result2 = model.fit_regularized(alpha=0.1, L1_wt=1)

# 정규화  result3 = Elastic Net 모형
result3 = model.fit_regularized(alpha=0.1, L1_wt=0.5)

# 5. test.csv load

In [55]:
test = pd.read_csv('../data/basic/test.csv', index_col = 0)

In [56]:
test.reset_index(inplace = True)

In [57]:
test.tail()

,date,store_nbr,item_nbr
526912,2014-10-26,45,107
526913,2014-10-26,45,108
526914,2014-10-26,45,109
526915,2014-10-26,45,110
526916,2014-10-26,45,111


## 5-1. test 데이터에 weather's feature merge후 test_df로 저장

In [79]:
test_df = test.merge(keys).merge(weather)

In [80]:
test_df.tail()

,date,store_nbr,item_nbr,station_nbr,tmax,tmin,tavg,depart,dewpoint,wetbulb,...,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,year,month,day,codesum
526912,2014-08-02,40,107,13,81.0,51.0,66.0,3.0,47.0,56.0,...,0.0,24.44,30.12,5.7,17.0,6.5,2014,8,2,
526913,2014-08-02,40,108,13,81.0,51.0,66.0,3.0,47.0,56.0,...,0.0,24.44,30.12,5.7,17.0,6.5,2014,8,2,
526914,2014-08-02,40,109,13,81.0,51.0,66.0,3.0,47.0,56.0,...,0.0,24.44,30.12,5.7,17.0,6.5,2014,8,2,
526915,2014-08-02,40,110,13,81.0,51.0,66.0,3.0,47.0,56.0,...,0.0,24.44,30.12,5.7,17.0,6.5,2014,8,2,
526916,2014-08-02,40,111,13,81.0,51.0,66.0,3.0,47.0,56.0,...,0.0,24.44,30.12,5.7,17.0,6.5,2014,8,2,


In [81]:
category = ['sunrise','sunset','codesum']

for each in category:
    test_df.drop(each,axis=1,inplace=True)
    
del test_df['tavg']
del test_df['sealevel']
del test_df['wetbulb']
del test_df['stnpressure']
del test_df['tmin']
del test_df['tmax']
del test_df['dewpoint']
del test_df['avgspeed']
del test_df['resultspeed']
del test_df['depart']
del test_df['heat']
del test_df['cool']
del test_df['snowfall']
del test_df['preciptotal']
del test_df['resultdir']


In [82]:
del test_df['month']

In [83]:
test_df.tail()

,date,store_nbr,item_nbr,station_nbr,year,day
526912,2014-08-02,40,107,13,2014,2
526913,2014-08-02,40,108,13,2014,2
526914,2014-08-02,40,109,13,2014,2
526915,2014-08-02,40,110,13,2014,2
526916,2014-08-02,40,111,13,2014,2


In [84]:
test_df = test_df.merge(codesum, on = ['date','store_nbr'])
# test_df['is_holiday'] = test_df['is_holiday'].apply(cate_holiday)
# test_df['codesum'] = test_df['codesum'].apply(cate_codesum)

## 5-2. test_df에서 팔린 units의 feature에 대해서만 선택후 test_X_df에 저장

In [85]:
test_X_df = pd.DataFrame(columns=['date','store_nbr', 'item_nbr', 'year', 'day','codesum'])

for nbr in range(1, 45+1):
    store_nbr = tmp[tmp['store_nbr'].isin([nbr])]
    sold_item_list = list(store_nbr['item_nbr'].unique())
    sold_item_list

    test_store_nbr = test_df[test_df['store_nbr'].isin([nbr])]
    to_test_df = test_store_nbr[test_store_nbr['item_nbr'].isin(sold_item_list)]

    test_X_df = pd.concat([test_X_df, to_test_df], axis = 0)
    
test_X_df.shape

(25444, 9)

In [86]:
test_X_df.head()

,codesum,date,day,dayofweek,is_holiday,item_nbr,station_nbr,store_nbr,year
484301,MO,2013-06-04,4,Tue,workday,9,1.0,1,2013
484320,MO,2013-06-04,4,Tue,workday,28,1.0,1,2013
484332,MO,2013-06-04,4,Tue,workday,40,1.0,1,2013
484339,MO,2013-06-04,4,Tue,workday,47,1.0,1,2013
484343,MO,2013-06-04,4,Tue,workday,51,1.0,1,2013


## 5-3. test_df와 type을 맞춰줘야함 그래야 predict()에 넣을 수 있다.

In [87]:
test_X_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25444 entries, 484301 to 514074
Data columns (total 9 columns):
codesum        25444 non-null object
date           25444 non-null object
day            25444 non-null object
dayofweek      25444 non-null object
is_holiday     25444 non-null object
item_nbr       25444 non-null object
station_nbr    25444 non-null float64
store_nbr      25444 non-null object
year           25444 non-null object
dtypes: float64(1), object(8)
memory usage: 1.9+ MB


In [88]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 526917 entries, 0 to 526916
Data columns (total 9 columns):
date           526917 non-null object
store_nbr      526917 non-null int64
item_nbr       526917 non-null int64
station_nbr    526917 non-null int64
year           526917 non-null int64
day            526917 non-null int64
is_holiday     526917 non-null object
codesum        526917 non-null object
dayofweek      526917 non-null object
dtypes: int64(5), object(4)
memory usage: 40.2+ MB


In [89]:
test_X_df['store_nbr'] = test_X_df['store_nbr'].astype(int)
test_X_df['item_nbr'] = test_X_df['item_nbr'].astype(int)
test_X_df['station_nbr'] = test_X_df['station_nbr'].astype(int)
test_X_df['year'] = test_X_df['year'].astype(int)
#test_X_df['month'] = test_X_df['month'].astype(int)
test_X_df['day'] = test_X_df['day'].astype(int)

In [90]:
test_X_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25444 entries, 484301 to 514074
Data columns (total 9 columns):
codesum        25444 non-null object
date           25444 non-null object
day            25444 non-null int64
dayofweek      25444 non-null object
is_holiday     25444 non-null object
item_nbr       25444 non-null int64
station_nbr    25444 non-null int64
store_nbr      25444 non-null int64
year           25444 non-null int64
dtypes: int64(5), object(4)
memory usage: 1.9+ MB


## 5-4. 이제 test_X_df 를 predict() 하면

### ridge모형
- result1

In [91]:
y_hat1 = result1.predict(test_X_df)

In [92]:
y_df1 = test_df.loc[:,['date','store_nbr','item_nbr']]

In [93]:
y_df1['units'] = y_hat1

In [94]:
y_df1.tail()

,date,store_nbr,item_nbr,units
526912,2014-08-02,40,107,NaN
526913,2014-08-02,40,108,NaN
526914,2014-08-02,40,109,NaN
526915,2014-08-02,40,110,NaN
526916,2014-08-02,40,111,NaN


In [95]:
def itiszero(units):
    if units > 0:
        return units
    else:
        return 0

In [96]:
y_df1['units'] = y_df1['units'].apply(itiszero)
y_df1['units'] = y_df1['units'].astype(int)

In [97]:
y_df1.head()

,date,store_nbr,item_nbr,units
0,2013-04-01,2,1,0
1,2013-04-01,2,2,0
2,2013-04-01,2,3,0
3,2013-04-01,2,4,0
4,2013-04-01,2,5,29


### Lasso 모형
- result2

In [98]:
y_hat2 = result2.predict(test_X_df)
y_df2 = test_df.loc[:,['date','store_nbr','item_nbr']]
y_df2['units'] = y_hat2
y_df2.tail()

y_df2['units'] = y_df2['units'].apply(itiszero)
y_df2['units'] = y_df2['units'].astype(int)
y_df2.head()

,date,store_nbr,item_nbr,units
0,2013-04-01,2,1,0
1,2013-04-01,2,2,0
2,2013-04-01,2,3,0
3,2013-04-01,2,4,0
4,2013-04-01,2,5,37


### Elastic Net 모형
- result3

In [99]:
y_hat3 = result3.predict(test_X_df)
y_df3 = test_df.loc[:,['date','store_nbr','item_nbr']]
y_df3['units'] = y_hat3
y_df3.tail()

y_df3['units'] = y_df3['units'].apply(itiszero)
y_df3['units'] = y_df3['units'].astype(int)
y_df3.head()

,date,store_nbr,item_nbr,units
0,2013-04-01,2,1,0
1,2013-04-01,2,2,0
2,2013-04-01,2,3,0
3,2013-04-01,2,4,0
4,2013-04-01,2,5,32


In [100]:
y_hat = result.predict(test_X_df)

In [101]:
y_df = test_df.loc[:,['date','store_nbr','item_nbr']]

In [102]:
y_df['units'] = y_hat

In [103]:
y_df.tail()

,date,store_nbr,item_nbr,units
526912,2014-08-02,40,107,NaN
526913,2014-08-02,40,108,NaN
526914,2014-08-02,40,109,NaN
526915,2014-08-02,40,110,NaN
526916,2014-08-02,40,111,NaN


In [104]:
def itiszero(units):
    if units > 0:
        return units
    else:
        return 0

In [105]:
y_df['units'] = y_df['units'].apply(itiszero)

In [106]:
y_df['units'] = y_df['units'].astype(int)

In [107]:
y_df

,date,store_nbr,item_nbr,units
0,2013-04-01,2,1,0
1,2013-04-01,2,2,0
2,2013-04-01,2,3,0
3,2013-04-01,2,4,0
4,2013-04-01,2,5,54
5,2013-04-01,2,6,0
6,2013-04-01,2,7,0
7,2013-04-01,2,8,0
8,2013-04-01,2,9,0
9,2013-04-01,2,10,0


# 6. 제출 형식에 맞게 수정

### 정규화 X

In [108]:
y_df['id'] = y_df['store_nbr'].astype(str)+"_"+y_df['item_nbr'].astype(str)+'_'+y_df['date'].astype(str)

In [109]:
for_submission = y_df.loc[:,['id','units']]

In [110]:
for_submission.head()

,id,units
0,2_1_2013-04-01,0
1,2_2_2013-04-01,0
2,2_3_2013-04-01,0
3,2_4_2013-04-01,0
4,2_5_2013-04-01,54


In [111]:
for_submission.to_csv('../model_generation/submission_csv/submission.csv', index = False)

### result1

In [112]:
y_df1['id'] = y_df1['store_nbr'].astype(str)+"_"+y_df1['item_nbr'].astype(str)+'_'+y_df1['date'].astype(str)

In [113]:
for_submission1 = y_df1.loc[:,['id','units']]

In [114]:
for_submission1.head()

,id,units
0,2_1_2013-04-01,0
1,2_2_2013-04-01,0
2,2_3_2013-04-01,0
3,2_4_2013-04-01,0
4,2_5_2013-04-01,29


In [115]:
for_submission1.to_csv('../model_generation/submission_csv/submission1.csv', index = False)

### result2

In [116]:
y_df2['id'] = y_df2['store_nbr'].astype(str)+"_"+y_df2['item_nbr'].astype(str)+'_'+y_df2['date'].astype(str)

In [117]:
for_submission2 = y_df2.loc[:,['id','units']]

In [118]:
for_submission2.head()

,id,units
0,2_1_2013-04-01,0
1,2_2_2013-04-01,0
2,2_3_2013-04-01,0
3,2_4_2013-04-01,0
4,2_5_2013-04-01,37


In [119]:
for_submission2.to_csv('../model_generation/submission_csv/submission2.csv', index = False)

### result3

In [120]:
y_df3['id'] = y_df3['store_nbr'].astype(str)+"_"+y_df3['item_nbr'].astype(str)+'_'+y_df3['date'].astype(str)

In [121]:
for_submission3 = y_df3.loc[:,['id','units']]

In [122]:
for_submission3.head()

,id,units
0,2_1_2013-04-01,0
1,2_2_2013-04-01,0
2,2_3_2013-04-01,0
3,2_4_2013-04-01,0
4,2_5_2013-04-01,32


In [123]:
for_submission3.to_csv('../model_generation/submission_csv/submission3.csv', index = False)